In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zoskoosp
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zoskoosp
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-h6bhnay2/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpsuhy3bfq".


In [ ]:
%%cuda

#include <stdio.h>

#define N 1000

__global__ void VectAdd(int *a, int *b, int *c){

      int indice = threadIdx.x + blockIdx.x * blockDim.x;

      if (indice < N){

          c[indice] = a[indice] + b[indice];

      }

}

int main(){

      int *a, *b, *c;

      int *gpu_a, *gpu_b, *gpu_c;

      int size = N*sizeof(int);

      a = (int *) malloc(size);

      b = (int *) malloc(size);

      c = (int *) malloc(size);

      for (int i = 0; i < N; i++){

          a[i] = i + 1;
          b[i] = i - 1;

      }

      cudaMalloc((void**) &gpu_a, size);
      cudaMalloc((void**) &gpu_b, size);
      cudaMalloc((void**) &gpu_c, size);

      cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
      cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

      VectAdd<<<2,512>>>(gpu_a, gpu_b, gpu_c);

      printf("Result: \n");

      cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);

      for (int i = 0; i < N; i++){

          printf("Element[%d] = %d\n", i, c[i]);

      }

      cudaFree(gpu_a);
      cudaFree(gpu_b);
      cudaFree(gpu_c);

      free(a);
      free(b);
      free(c);

      return 0;

}

Result: 
Element[0] = 0
Element[1] = 2
Element[2] = 4
Element[3] = 6
Element[4] = 8
Element[5] = 10
Element[6] = 12
Element[7] = 14
Element[8] = 16
Element[9] = 18
Element[10] = 20
Element[11] = 22
Element[12] = 24
Element[13] = 26
Element[14] = 28
Element[15] = 30
Element[16] = 32
Element[17] = 34
Element[18] = 36
Element[19] = 38
Element[20] = 40
Element[21] = 42
Element[22] = 44
Element[23] = 46
Element[24] = 48
Element[25] = 50
Element[26] = 52
Element[27] = 54
Element[28] = 56
Element[29] = 58
Element[30] = 60
Element[31] = 62
Element[32] = 64
Element[33] = 66
Element[34] = 68
Element[35] = 70
Element[36] = 72
Element[37] = 74
Element[38] = 76
Element[39] = 78
Element[40] = 80
Element[41] = 82
Element[42] = 84
Element[43] = 86
Element[44] = 88
Element[45] = 90
Element[46] = 92
Element[47] = 94
Element[48] = 96
Element[49] = 98
Element[50] = 100
Element[51] = 102
Element[52] = 104
Element[53] = 106
Element[54] = 108
Element[55] = 110
Element[56] = 112
Element[57] = 114
Element[58] 

In [ ]:
%%cuda

#include <stdio.h>

#define N 4

__global__ void ProduitScalaire(int *a, int *b, int *c){

      __shared__ int temp[N];

      int indice = threadIdx.x;

      if (indice < N){

        temp[indice] = a[indice] * b[indice];

      }

      __syncthreads();

      if (indice == 0){

          int sum = 0;

          for (int i = 0; i < N; i++){

              sum = sum + temp[i];

          }

          (*c) = sum;

      }

}

int main(){

      int *a, *b, *c;

      int *gpu_a, *gpu_b, *gpu_c;

      int size = N*sizeof(int);

      a = (int *) malloc(size);

      b = (int *) malloc(size);

      c = (int *) malloc(sizeof(int));

      for (int i = 0; i < N; i++){

          a[i] = i;
          b[i] = 2*i;

      }

      cudaMalloc((void**) &gpu_a, size);
      cudaMalloc((void**) &gpu_b, size);
      cudaMalloc((void**) &gpu_c, sizeof(int));

      cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
      cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

      ProduitScalaire<<<1,512>>>(gpu_a, gpu_b, gpu_c);

      printf("Result: \n");

      cudaMemcpy(c, gpu_c, sizeof(int), cudaMemcpyDeviceToHost);

      printf("Le produit scalaire vaut: %d.\n", (*c));

      cudaFree(gpu_a);
      cudaFree(gpu_b);
      cudaFree(gpu_c);

      free(a);
      free(b);
      free(c);

      return 0;

}

Result: 
Le produit scalaire vaut: 28.



In [ ]:
%%cuda

#include <stdio.h>

#define N 2048

__global__ void ProduitScalaire(int *a, int *b, int *c){

      __shared__ int temp[N];

      int indice = threadIdx.x + blockIdx.x * blockDim.x;

      if (indice < N){

        temp[indice] = a[indice] * b[indice];

      }

      __syncthreads();

      if (indice == blockIdx.x * blockDim.x){

          int sum = 0;

          for (int i = 0; i < N; i++){

              sum = sum + temp[i];

          }

          atomicAdd(c, sum);

      }

}

int main(){

      int *a, *b, *c;

      int *gpu_a, *gpu_b, *gpu_c;

      int size = N*sizeof(int);

      a = (int *) malloc(size);

      b = (int *) malloc(size);

      c = (int *) malloc(sizeof(int));

      for (int i = 0; i < N; i++){

          a[i] = i;
          b[i] = 2*i;

      }

      cudaMalloc((void**) &gpu_a, size);
      cudaMalloc((void**) &gpu_b, size);
      cudaMalloc((void**) &gpu_c, sizeof(int));

      cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
      cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

      ProduitScalaire<<<4,512>>>(gpu_a, gpu_b, gpu_c);

      printf("Result: \n");

      cudaMemcpy(c, gpu_c, sizeof(int), cudaMemcpyDeviceToHost);

      printf("Le produit scalaire vaut: %d.\n", (*c));

      cudaFree(gpu_a);
      cudaFree(gpu_b);
      cudaFree(gpu_c);

      free(a);
      free(b);
      free(c);

      return 0;

}

Result: 
Le produit scalaire vaut: 1427462144.



In [ ]:
%%cuda

#include <stdio.h>

#define N 2048

__global__ void PermuterVecteur(int *a, int *c){

      int indice = threadIdx.x + blockIdx.x * blockDim.x;

      if (indice < N){

          c[indice] = a[N - 1 - indice];
      }

}

int main(){

      int *a, *c;

      int *gpu_a, *gpu_c;

      int size = N*sizeof(int);

      a = (int *) malloc(size);
      c = (int *) malloc(size);

      for (int i = 0; i < N; i++){

          a[i] = i + 1;

      }

      cudaMalloc((void**) &gpu_a, size);
      cudaMalloc((void**) &gpu_c, size);

      cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);

      PermuterVecteur<<<4,512>>>(gpu_a, gpu_c);

      printf("Result: \n");

      cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);

      for (int i = 0; i < N; i++){

          printf("Element[%d] = %d\n", i, c[i]);

      }

      cudaFree(gpu_a);
      cudaFree(gpu_c);

      free(a);
      free(c);

      return 0;

}

Result: 
Element[0] = 2048
Element[1] = 2047
Element[2] = 2046
Element[3] = 2045
Element[4] = 2044
Element[5] = 2043
Element[6] = 2042
Element[7] = 2041
Element[8] = 2040
Element[9] = 2039
Element[10] = 2038
Element[11] = 2037
Element[12] = 2036
Element[13] = 2035
Element[14] = 2034
Element[15] = 2033
Element[16] = 2032
Element[17] = 2031
Element[18] = 2030
Element[19] = 2029
Element[20] = 2028
Element[21] = 2027
Element[22] = 2026
Element[23] = 2025
Element[24] = 2024
Element[25] = 2023
Element[26] = 2022
Element[27] = 2021
Element[28] = 2020
Element[29] = 2019
Element[30] = 2018
Element[31] = 2017
Element[32] = 2016
Element[33] = 2015
Element[34] = 2014
Element[35] = 2013
Element[36] = 2012
Element[37] = 2011
Element[38] = 2010
Element[39] = 2009
Element[40] = 2008
Element[41] = 2007
Element[42] = 2006
Element[43] = 2005
Element[44] = 2004
Element[45] = 2003
Element[46] = 2002
Element[47] = 2001
Element[48] = 2000
Element[49] = 1999
Element[50] = 1998
Element[51] = 1997
Element[52] =

In [ ]:
%%cuda

#include <stdio.h>

#define N 128

#define GRIDSIZE 16

__global__ void ProduitScalaire(int *a, int *b, int *c){

      __shared__ int temp[N];

      int indice = threadIdx.y * GRIDSIZE + threadIdx.x;

      if (indice < N){

        temp[indice] = a[indice] * b[indice];

      }

      __syncthreads();

      if (indice == 0){

          int sum = 0;

          for (int i = 0; i < N; i++){

              sum = sum + temp[i];

          }

          (*c) = sum;

      }

}

int main(){

      int *a, *b, *c;

      int *gpu_a, *gpu_b, *gpu_c;

      int size = N*sizeof(int);

      a = (int *) malloc(size);

      b = (int *) malloc(size);

      c = (int *) malloc(sizeof(int));

      for (int i = 0; i < N; i++){

          a[i] = i;
          b[i] = 2*i;

      }

      cudaMalloc((void**) &gpu_a, size);
      cudaMalloc((void**) &gpu_b, size);
      cudaMalloc((void**) &gpu_c, sizeof(int));

      cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
      cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

      dim3 grid(GRIDSIZE, GRIDSIZE);

      ProduitScalaire<<<1,grid>>>(gpu_a, gpu_b, gpu_c);

      printf("Result: \n");

      cudaMemcpy(c, gpu_c, sizeof(int), cudaMemcpyDeviceToHost);

      printf("Le produit scalaire vaut: %d.\n", (*c));

      cudaFree(gpu_a);
      cudaFree(gpu_b);
      cudaFree(gpu_c);

      free(a);
      free(b);
      free(c);

      return 0;

}

Result: 
Le produit scalaire vaut: 1381760.



In [ ]:
%%cuda

#include <stdlib.h>
#include <stdio.h>

__global__ void operation(int *a, int* b, int* c, int *d){

    d[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x] * c[threadIdx.x];

}

int main(){

    int *a, *b, *c, *d, *gpu_a, *gpu_b, *gpu_c, *gpu_d;

    a = (int *)malloc(64*sizeof(int));

    b = (int *)malloc(64*sizeof(int));

    c = (int *)malloc(64*sizeof(int));

    d = (int *)malloc(64*sizeof(int));

    for (int i = 0; i < 64; i++){

        a[i] = i + 1;
        b[i] = i*2;
        c[i] = i - 1;

    }

    cudaMalloc((void **) &gpu_a, 64*sizeof(int));

    cudaMalloc((void **) &gpu_b, 64*sizeof(int));

    cudaMalloc((void **) &gpu_c, 64*sizeof(int));

    cudaMalloc((void **) &gpu_d, 64*sizeof(int));

    cudaMemcpy(gpu_a, a, 64*sizeof(int), cudaMemcpyHostToDevice);

    cudaMemcpy(gpu_b, b, 64*sizeof(int), cudaMemcpyHostToDevice);

    cudaMemcpy(gpu_c, c, 64*sizeof(int), cudaMemcpyHostToDevice);

    operation<<<1,64>>>(gpu_a, gpu_b, gpu_c, gpu_d);

    cudaMemcpy(d, gpu_d, 64*sizeof(int), cudaMemcpyHostToDevice);

    cudaDeviceSynchronize();

    cudaFree(gpu_a); cudaFree(gpu_b); cudaFree(gpu_d); cudaFree(gpu_d);

    free(a); free(b); free(c); free(d);

    return 0;

}

In [ ]:
%%cuda

#include <stdlib.h>
#include <stdio.h>

#define N 100

__global__ void kernel(int *tab, int *result){

    int indice = threadIdx.x + blockIdx.x * blockDim.x;

    int sum = 0;

    if (indice < N){

        for (int i = 0; i <= indice; i++){

            sum = sum + tab[i];

        }

        result[indice] = sum ;

    }

}

int main(){

    int *tab, *result, *gpu_tab, *result_gpu;

    tab = (int *) malloc(N*sizeof(int));

    result = (int *) malloc(N*sizeof(int));

    for (int i = 0; i < N; i++){

        tab[i] = i + 1;

    }

    cudaMalloc((void**)&gpu_tab, N*sizeof(int));

    cudaMalloc((void**)&result_gpu, N*sizeof(int));

    cudaMemcpy(gpu_tab, tab, N*sizeof(int), cudaMemcpyHostToDevice);

    kernel<<<7,16>>>(gpu_tab, result_gpu);

    cudaDeviceSynchronize();

    cudaMemcpy(result, result_gpu, N*sizeof(int), cudaMemcpyDeviceToHost);

    for (int i = 0; i < N; i++){

        printf("Élement[%d] == %d\n", i, result[i]);

    }

    free(tab);

    free(result);

    cudaFree(gpu_tab);

    cudaFree(result_gpu);

    return 0;

}

Élement[0] == 1
Élement[1] == 3
Élement[2] == 6
Élement[3] == 10
Élement[4] == 15
Élement[5] == 21
Élement[6] == 28
Élement[7] == 36
Élement[8] == 45
Élement[9] == 55
Élement[10] == 66
Élement[11] == 78
Élement[12] == 91
Élement[13] == 105
Élement[14] == 120
Élement[15] == 136
Élement[16] == 153
Élement[17] == 171
Élement[18] == 190
Élement[19] == 210
Élement[20] == 231
Élement[21] == 253
Élement[22] == 276
Élement[23] == 300
Élement[24] == 325
Élement[25] == 351
Élement[26] == 378
Élement[27] == 406
Élement[28] == 435
Élement[29] == 465
Élement[30] == 496
Élement[31] == 528
Élement[32] == 561
Élement[33] == 595
Élement[34] == 630
Élement[35] == 666
Élement[36] == 703
Élement[37] == 741
Élement[38] == 780
Élement[39] == 820
Élement[40] == 861
Élement[41] == 903
Élement[42] == 946
Élement[43] == 990
Élement[44] == 1035
Élement[45] == 1081
Élement[46] == 1128
Élement[47] == 1176
Élement[48] == 1225
Élement[49] == 1275
Élement[50] == 1326
Élement[51] == 1378
Élement[52] == 1431
Élement[53